<a href="https://colab.research.google.com/github/manjotmb20/EC3/blob/master/Spambase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from numpy import linalg as LA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
nltk.download("punkt")
# Warnings
import warnings
warnings.filterwarnings('ignore')

# Styles
plt.style.use('ggplot')
sns.set_style('whitegrid')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12
plt.rcParams['patch.force_edgecolor'] = True

# Text Preprocessing
import nltk
nltk.download("all")
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load("en")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora

In [2]:
messages = pd.read_csv("spam.csv", encoding = 'latin-1')

# Drop the extra columns and rename columns

messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["category", "text"]
topMessages = messages.groupby("text")["category"].agg([len, np.max]).sort_values(by = "len", ascending = False).head(n = 10)
display(topMessages)

,len,amax
text,,
"Sorry, I'll call later",30,ham
I cant pick the phone right now. Pls send a message,12,ham
Ok...,10,ham
Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..,4,ham
"Wen ur lovable bcums angry wid u, dnt take it seriously.. Coz being angry is d most childish n true way of showing deep affection, care n luv!.. kettoda manda... Have nice day da.",4,ham
Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£1000 cash or å£5000 prize!,4,spam
Okie,4,ham
"Say this slowly.? GOD,I LOVE YOU &amp; I NEED YOU,CLEAN MY HEART WITH YOUR BLOOD.Send this to Ten special people &amp; u c miracle tomorrow, do it,pls,pls do it...",4,ham
"7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st \Ur Lovely Friendship\""... good morning dear""",4,ham


In [3]:
spam_messages = messages[messages["category"] == "spam"]["text"]
ham_messages = messages[messages["category"] == "ham"]["text"]

spam_words = []
ham_words = []

# Since this is just classifying the message as spam or ham, we can use isalpha(). 
# This will also remove the not word in something like can't etc. 
# In a sentiment analysis setting, its better to use 
# sentence.translate(string.maketrans("", "", ), chars_to_remove)

def extractSpamWords(spamMessages):
    global spam_words
    words = [word.lower() for word in word_tokenize(spamMessages) if word.lower() not in stopwords.words("english") and word.lower().isalpha()]
    spam_words = spam_words + words
    
def extractHamWords(hamMessages):
    global ham_words
    words = [word.lower() for word in word_tokenize(hamMessages) if word.lower() not in stopwords.words("english") and word.lower().isalpha()]
    ham_words = ham_words + words

spam_messages.apply(extractSpamWords)
ham_messages.apply(extractHamWords)

0       None
1       None
3       None
4       None
6       None
        ... 
5565    None
5568    None
5569    None
5570    None
5571    None
Name: text, Length: 4825, dtype: object

In [4]:
spam_words = np.array(spam_words)
print("Top 10 Spam words are :\n")
pd.Series(spam_words).value_counts().head(n = 10)

Top 10 Spam words are :



call      346
free      217
txt       156
u         144
ur        144
mobile    123
text      121
stop      114
claim     113
reply     104
dtype: int64

In [5]:
ham_words = np.array(ham_words)
print("Top 10 Ham words are :\n")
pd.Series(ham_words).value_counts().head(n = 10)

Top 10 Ham words are :



u       974
gt      318
lt      316
get     301
go      246
ok      246
got     242
ur      237
know    234
like    231
dtype: int64

In [6]:
messages["messageLength"] = messages["text"].apply(len)
messages["messageLength"].describe()

count    5572.000000
mean       80.118808
std        59.690841
min         2.000000
25%        36.000000
50%        61.000000
75%       121.000000
max       910.000000
Name: messageLength, dtype: float64

In [7]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

def cleanText(message):
    
    message = message.translate(str.maketrans('', '', string.punctuation))
    words = [stemmer.stem(word) for word in message.split() if word.lower() not in stopwords.words("english")]
    
    return " ".join(words)

messages["text"] = messages["text"].apply(cleanText)
messages.head(n = 10)    

,category,text,messageLength
0,ham,go jurong point crazi avail bugi n great world...,111
1,ham,ok lar joke wif u oni,29
2,spam,free entri 2 wkli comp win fa cup final tkts 2...,155
3,ham,u dun say earli hor u c alreadi say,49
4,ham,nah dont think goe usf live around though,61
5,spam,freemsg hey darl 3 week word back id like fun ...,148
6,ham,even brother like speak treat like aid patent,77
7,ham,per request mell mell oru minnaminungint nurun...,160
8,spam,winner valu network custom select receivea å£9...,158
9,spam,mobil 11 month u r entitl updat latest colour ...,154


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(encoding = "latin-1", strip_accents = "unicode", stop_words = "english")
features = vec.fit_transform(messages["text"])
print(features.shape)

(5572, 7903)


In [0]:
def encodeCategory(cat):
    if cat == "spam":
        return 1
    else:
        return 0
        
messages["category"] = messages["category"].apply(encodeCategory)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, messages["category"], stratify = messages["category"], test_size = 0.2)


In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score

from sklearn.naive_bayes import MultinomialNB
gaussianNb = MultinomialNB()
gaussianNb.fit(X_train, y_train)


y_pred = gaussianNb.predict(X_test)
c=gaussianNb.predict_proba(X_test)
print(fbeta_score(y_test, y_pred, beta = 0.5))
print("Accuracy value for LR",roc_auc_score(y_test,y_pred))

0.9203539823008849
Accuracy value for LR 0.848993288590604


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
classifier3=SVC(probability=True)
from sklearn.metrics import roc_auc_score,accuracy_score
classifier=LogisticRegression(random_state=0)
classifier.fit(X_train,y_train)
ypred=classifier.predict(X_test)
print("Accuracy score of LR",accuracy_score(y_test,ypred))
y_prob=classifier.predict_proba(X_test)
df=pd.DataFrame()
df['pred']=ypred
survived=pd.get_dummies(df.pred,prefix='Survived')
df=pd.concat([df,survived],axis=1)
df.drop('pred',inplace=True,axis=1)
df2=pd.DataFrame()
classifier3.fit(X_train,y_train)
ypred3=classifier3.predict(X_test)
print("Accuracy score of SVM",accuracy_score(y_test,ypred3))
'''df2['pred']=ypred3'''
y_prob3=classifier3.predict_proba(X_test)
'''survived2=pd.get_dummies(df2.pred,prefix='Surviveds')
df=pd.concat([df,survived2],axis=1)'''
df_kmeans=pd.DataFrame()
df_kmeans['Survived_k']=y_pred
survivedk=pd.get_dummies(df_kmeans.Survived_k,prefix='Survivedk')
df['Survivedk_0']=survivedk['Survivedk_0']
df['Survivedk_1']=survivedk['Survivedk_1']
print("Accuracy score of SVM",roc_auc_score(y_test,y_pred))

Accuracy score of LR 0.9497757847533632
Accuracy score of SVM 0.8663677130044843
Accuracy score of SVM 0.848993288590604


In [18]:
print("Accuracy value for LR",accuracy_score(y_test,ypred))
print("AUC for for LR",roc_auc_score(y_test,ypred))

Accuracy value for LR 0.9497757847533632
AUC for for LR 0.8120805369127517


**Generating normalized matrices Km and Kc using the method mentioned in paper 2.**

In [0]:
m=np.array(df)
s=np.zeros(shape=(m.shape[0],m.shape[0]))
for i in range(m.shape[0]):
  for j in range(m.shape[0]):
    if i!=j:
      sum1=0
      for k in range(2,m.shape[1]):
        sum1=sum1+(m[i][k]*m[j][k])
      s[i][j]=sum1
w=s
d=np.zeros(shape=w.shape[0])
for i in range(d.shape[0]):
  d[i]=np.sum(w[i],axis=0)
wnew=w/d.reshape(d.shape[0],1)
Km=wnew     

**Checking whether the matrices have been normalized or not i.e making them bistochastic **

In [24]:
print(np.sum(Km,axis=1))
print(np.sum(Km,axis=0))

[1. 1. 1. ... 1. 1. 1.]
[1. 1. 1. ... 1. 1. 1.]


Similarly for Kc we generate a bistochastic matrix where m=the co-occurence matrix.

In [26]:
m=np.array(df)
s=np.zeros(shape=(m.shape[0],m.shape[0]))
for i in range(m.shape[0]):
  for j in range(m.shape[0]):
    if i!=j:
      sum1=0
      for k in range(2,m.shape[1]):
        sum1=sum1+(m[i][k]*m[j][k])
      s[i][j]=sum1
m=s   
s=np.zeros(shape=(m.shape[0],m.shape[0]))
for i in range(m.shape[0]):
  for j in range(m.shape[0]):
    if i!=j:
      sum1=0
      for k in range(2,m.shape[1]):
        sum1=sum1+(m[i][k]*m[j][k])
      s[i][j]=sum1   
d=np.zeros(shape=w.shape[0])
w=s
for i in range(d.shape[0]):
  d[i]=np.sum(w[i],axis=0)
wnew=w/d.reshape(d.shape[0],1)  
Kc=wnew
Kc

KeyboardInterrupt: ignored

**Same, checking for bi-stochastic behaviour**

In [27]:
  np.sum(Kc,axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

**Generating Fg and Fo i'e the Group class matrix and Object Class matrix**

In [28]:

objclass_mat1=pd.DataFrame(y_prob,columns=['Survived','NotSurvived'])
objclass_mat2=pd.DataFrame(y_prob3,columns=['Survived','NotSurvived'])
df_t=pd.DataFrame(objclass_mat1)
Fo=np.array(df_t)
gpclass_mat=pd.DataFrame(columns=['Survived','NotSurvived'],index=['class1surv','class1notsurv','class2surv','class2notsurv'])
prob1s=np.sum(objclass_mat1.Survived)/len(objclass_mat1)
prob1ns=np.sum(objclass_mat1.NotSurvived)/len(objclass_mat1)
prob2s=np.sum(objclass_mat2.Survived)/len(objclass_mat2)
prob2ns=np.sum(objclass_mat2.NotSurvived)/len(objclass_mat2)
list1=[]
list1.append(prob1s)
list1.append(prob1ns)
list1.append(prob2s)
list1.append(prob2ns)

list2=[]
list2.append(prob1ns)
list2.append(prob1s)
list2.append(prob2ns)
list2.append(prob2s)
gpclass_mat['Survived']=list1
gpclass_mat['NotSurvived']=list2
gpclass_mat
Fg=np.array(gpclass_mat)
Fg

array([[0.86961239, 0.13038761],
       [0.13038761, 0.86961239],
       [0.8663114 , 0.1336886 ],
       [0.1336886 , 0.8663114 ]])

**Similarly, generating  Yo and Yg**

In [0]:
n=2
C=survived.shape[1]
G=survived.shape[1]*n

Yo=np.zeros(shape=(df.shape[0],C))
cal_y=np.array(df)
for i in range(Yo.shape[0]):
  Yo[i][0]=(cal_y[i][0]+cal_y[i][2])/C
for i in range(Yo.shape[0]):
  Yo[i][1]=(cal_y[i][1]+cal_y[i][3])/C

In [30]:
G

4

In [31]:
m=np.array(df)
Yg=np.zeros(shape=(m.shape[1],2))
c=np.sum(m,axis=0)
for i in range(Yg.shape[0]):
  if i%2==0:
    Yg[i][0]=c[i]/m.shape[0]
    Yg[i][1]=1-Yg[i][0]
  else:
    Yg[i][1]=c[i]/m.shape[0]
    Yg[i][0]=1-Yg[i][1]  
Yg

array([[0.91659193, 0.08340807],
       [0.91659193, 0.08340807],
       [0.90672646, 0.09327354],
       [0.90672646, 0.09327354]])

Genearting 

Checking for the conditions specified in eq 6

In [32]:
Fo_cond=True
Fg_cond=True
for i in range(Fo.shape[0]):
  if np.linalg.norm((Fo[i]), ord=1)!=1:
    Fo_cond=False
for i in range(Fg.shape[0]):
  if np.linalg.norm((Fg[i]), ord=1)!=1:
    Fo_cond=False    
print(Fo_cond)
print(Fg_cond)

True
True


**Generating diagonal matrices Dm and Dc**

In [33]:
test_dm=np.zeros(shape=(Km.shape))
test_Km=Km
for i in range(test_Km.shape[0]):
  test_dm[i]=np.sum(test_Km[i],axis=0)
Dm=np.diag(np.diag(test_dm))
print("Shape of DM",Dm.shape)
test_dc=np.zeros(shape=(Kc.shape))
test_Kc=Kc
for i in range(test_Kc.shape[0]):
  test_dc[i]=np.sum(test_Kc[i],axis=0)
Dc=np.diag(np.diag(test_dc))
print("shape of Dc",Dc.shape)

Shape of DM (1115, 1115)
shape of Dc (1115, 1115)


In [34]:
print("KHM",Km.shape)
print("KHC",Kc.shape)
print("Fo",Fo.shape)
print("Fg",Fg.shape)
print("Yo",Yo.shape)
print("Yg",Yg.shape)
print("Dm",Dm.shape)
print("Dc",Dc.shape)

KHM (1115, 1115)
KHC (1115, 1115)
Fo (1115, 2)
Fg (4, 2)
Yo (1115, 2)
Yg (4, 2)
Dm (1115, 1115)
Dc (1115, 1115)


In [35]:
Yg=np.pad(Yg,pad_width=[(0, Km.shape[0]-G), (0, 0)])
Yg.shape

(1115, 2)

In [36]:
print("KHM",Km.shape)
print("KHC",Kc.shape)
print("Fo",Fo.shape)
print("Fg",Fg.shape)
print("Yo",Yo.shape)
print("Yg",Yg.shape)
print("Dm",Dm.shape)
print("Dc",Dc.shape)

KHM (1115, 1115)
KHC (1115, 1115)
Fo (1115, 2)
Fg (4, 2)
Yo (1115, 2)
Yg (1115, 2)
Dm (1115, 1115)
Dc (1115, 1115)


In [0]:
import sys
threshold=sys.float_info.epsilon

**Implementing algorithm 2**

In [0]:
alpha=0.40
beta=0.35
gamma=0.45
delta=0.5
Fo_t=np.zeros(shape=Fo.shape)
Fg_t=np.zeros(shape=Fg.shape)
Fot_old=np.zeros(shape=Fo.shape)
Fo_t=Fo
Fg_t=Fg
while LA.norm((Fo_t-Fot_old),'fro')>threshold:
  Fot_old=Fo_t
  Fg_t=np.matmul((np.linalg.inv(2*delta*np.ones(shape=Dm.shape)+(alpha*Dm))),((alpha*np.matmul(Km,Fo_t))+(2*delta*Yg)))
  Fo_t=np.matmul((np.linalg.inv((alpha*Dm)+(2*beta*Dc)-(beta*np.matmul(np.identity(Kc.shape[0]),Kc))-(beta*np.matmul(np.ones(shape=Kc.shape),Kc))+(2*gamma*np.identity(Kc.shape[0])))),((alpha*np.matmul(Km,Fg_t))+(2*gamma*Yo)))

In [41]:
from sklearn.metrics import accuracy_score
result=np.zeros(shape=Fo_t.shape[0])
for i in range(Fo_t.shape[0]):
  if Fo_t[i][0]<Fo_t[i][1]:
    result[i]=1
print("Accuracy value for Ensemble model",accuracy_score(y_test,result))
print("AUC for for Ensemble model",roc_auc_score(y_test,result))
print("Accuracy value for LR",accuracy_score(y_test,ypred))
print("AUC for for LR",roc_auc_score(y_test,ypred))



Accuracy value for Ensemble model 0.9668161434977578
AUC for for Ensemble model 0.8758389261744967
Accuracy value for LR 0.9497757847533632
AUC for for LR 0.8120805369127517


**We see that the ensemble model performs better than the Random forest, Kneigbours and Svm models and is at par with the Logistic regression model that gives us the best accuracy. **